# Unit of Work

The purpose of a unit of work is to group all our updates to the DB into a single 'unit' of updates. In relational DBs, this is often wrapped in a DB transaction, but we won't do that here....

In [1]:
from barin2.identity_map import IdentityMap
from barin2.instrumentation import get_state, Status, set_object_status

import pymongo
cli = pymongo.MongoClient('mongodb://mongo')
db = cli.barin2
idmap = IdentityMap()

In [2]:
user_doc = db.users.find_one()
user_mdoc = idmap.process(db.users, user_doc, Status.CLEAN)
user_mdoc

{'_id': ObjectId('615e1ff773b0c646ff5e2a57'),
 'name': 'Rick',
 'classes': ['Python', 'ML']}

In [3]:
get_state(user_mdoc)

<istate status=Status.CLEAN collection=barin2.users>

# Thinking about state

Documents (and fields) can have various states beyond dirty/not dirty

- clean - memory matches db
- new - in memory, not in db
- dirty - in both, memory has changes
- deleted - in db, not in memory

In [4]:
get_state(user_mdoc['classes'])

<istate status=Status.CLEAN container={'_id': ObjectId('615e1ff773b0c646ff5e2a57'), 'name': 'Rick', 'classes': ['Python', 'ML']}>

Check our changes

In [5]:
user_mdoc['business'] = 'Arborian'
get_state(user_mdoc)

Before modify
Copy object to _pristine


<istate status=Status.DIRTY collection=barin2.users>

In [6]:
set_object_status(user_mdoc, Status.CLEAN)
get_state(user_mdoc)

<istate status=Status.CLEAN collection=barin2.users>

In [7]:
user_mdoc.update(business='Arborian')
get_state(user_mdoc)

Before modify
Copy object to _pristine


<istate status=Status.DIRTY collection=barin2.users>

In [8]:
set_object_status(user_mdoc, Status.CLEAN)
user_mdoc['classes'].pop()
get_state(user_mdoc)

Before modify
Copy object to _pristine


<istate status=Status.DIRTY collection=barin2.users>

In [9]:
user_mdoc = idmap.process(db.users, user_doc, Status.CLEAN)
user_mdoc

{'_id': ObjectId('615e1ff773b0c646ff5e2a57'),
 'name': 'Rick',
 'classes': ['Python', 'ML']}

In [10]:
get_state(user_mdoc)

<istate status=Status.CLEAN collection=barin2.users>

In [11]:
get_state(user_mdoc['classes'])

<istate status=Status.CLEAN container={'_id': ObjectId('615e1ff773b0c646ff5e2a57'), 'name': 'Rick', 'classes': ['Python', 'ML']}>

In [12]:
def compute_update(obj):
    # Check for dirty Idict and Ilist values
    for key, value in obj.items():
        value_state = get_state(value)
        if not value_state:
            continue
        if value_state.status == Status.DIRTY:
            yield '$set', key, value
            
    # Check for changed primitive values
    pristine = get_state(obj).pristine
    if not pristine:
        # No primitives were modified
        return
    added = obj.keys() - pristine.keys()
    maybe_updated = obj.keys() & pristine.keys()
    deleted = pristine.keys() - obj.keys()
    for key in added:
        yield '$set', key, obj[key]
    for key in deleted:
        yield '$unset', key
    for key in maybe_updated:
        pristine_value = pristine[key]
        current_value = obj[key]
        value_state = get_state(current_value)
        if value_state:
            # Already handled these, above
            continue
        if pristine_value != current_value:
            yield '$set', key, current_value
    
def compute_changes(obj):
    state = get_state(obj)
    if state.status == Status.CLEAN:
        return
    elif state.status == Status.NEW:
        yield 'insert', obj
    elif state.status == Status.DELETED:
        yield 'delete', obj
    elif state.status == Status.DIRTY:
        yield from compute_update(obj)

In [13]:
user_mdoc['classes'].append('React')

Before modify
Copy object to _pristine


In [14]:
for item in compute_changes(user_mdoc):
    print(item)

('$set', 'classes', ['Python', 'ML', 'React'])


In [15]:
class UnitOfWork:
    def __init__(self, idmap):
        self._idmap = idmap
        
    def add(self, obj, collection=None):
        """Add (mark for insertion if new) an item"""
        return self._idmap.process(collection, obj, Status.NEW)
    
    def delete(self, obj, collection=None):
        """Remove (mark for deletion) an item"""
        return self._idmap.process(collection, obj, Status.DELETED)
    

# TODO

Basic use cases:

  - create dict, insert into UOW, flush UOW .insert()s doc
  - query doc, save to UOW, modify, flush UOW .update()s doc
  - query doc, save to UOW, delete from UOW .delete()s doc
